# ST7 Planification quotidienne d’une équipe mobile
# Phase III

## Importation de modules

In [2]:
# module importation
import numpy as np
from math import ceil
import matplotlib.pyplot as plt
import itertools

# utilities
from utils import *
from copy import deepcopy

# model classes for employees and nodes
from models_v2 import Employee, Node, Task, Home, Unavail

Déclaration des constantes et des indices :

In [3]:
W = U = T = V = 0
employees = homes = tasks = unavails = nodes = []

## Fonction de lecture de données

In [4]:
def load_data_from_path(path_to_instance: str):
    # load employee data
    Employee.load_excel(path_to_instance)

    # load node data
    Node.clear_previous_data()
    for cls in [Home, Task, Unavail]:
        cls.load_excel(path_to_instance)
    Node.initialize_distance()

    # constants
    global W, U, T, V
    (W, U, T, V) = (Task.count, Unavail.count, Employee.count, Node.count)

    # indices of employees, homes, lunches, tasks, unavailabilities
    global employees, homes, tasks, unavails, nodes
    employees = list(range(T))
    homes = list(range(T))
    tasks = list(range(T, T + W))
    unavails = list((range(T + W, V)))
    nodes = list(range(V))

Lecture de données de test

In [5]:
path_to_test = path_columbia_v3 = "./data/InstancesV3/InstanceColumbiaV3.xlsx"
load_data_from_path(path_to_test)

## Implémentation de la classe Solution

In [6]:
class Chemins:
    print_warning = True  # whether to print warning or not when validating

    def __init__(self,chemins):
        self.list = chemins
    
    @classmethod
    def set_warning(cls, print_warning: bool):
        """set whether to print warning when validating data"""
        cls.print_warning = print_warning

    @classmethod
    def warn(cls, warning: str, print_color="yellow"):
        """
        Print a text in yellow if print_warning set to True
        :param warning: the warning message we want to print
        :param print_color: the color of the printed message, defaulted to yellow
        """
        if not cls.print_warning: return # does nothing if print_warning set to False
        correspondance = {
            "yellow": Colors.WARNING,
            "cyan": Colors.CYAN,
            "green": Colors.GREEN,
            "red": Colors.FAIL
        }
        color = correspondance[print_color] if print_color in correspondance.keys() else Colors.WARNING
        print(f"{color}Error: {warning}{Colors.NORMAL}")

    def copy(self):
        """Return a deep copy of the instance"""
        return deepcopy(self)

    def validate_format(self):
        """Verify that chemins and margin have the right format"""
        if type(self.list) == str :
            Chemins.warn(self.list)
            return False

        for k in self.list.keys():
            if k not in employees :
                Chemins.warn(f"{k} is not the index of an employee")
                return False
            for v in list(self.list[k].keys())[1:] : 
                if v not in tasks + unavails: 
                    Chemins.warn(f"{v} is not the index of a task or an unvavailability")
                    return False
        return True

    def validate(self):
        """
        validate the modifications
        :return: whether the constraints are verified
        """

        # verify that x, y and l contain only 0 and 1
        if not self.validate_format(): return False

        # C1, okay thank to the format 

        # C2
        visited = []
        for k in self.list.keys():
            for v in self.list[k].keys():
                if v in visited :
                    Chemins.warn("C2: Each task should be done by at most one employee.")
                    Chemins.warn(f"Condition violated by node {v} done by employees {k}.", "cyan")
                    return False
                visited.append(v)

        # C3.a and C3.b2
        for v in unavails :
            for k in self.list.keys() :
                if Node.list[v].employee.index_of() == k and not v in self.list[k].keys():
                    Chemins.warn("C3.a: Unavailabilities should be visited.")
                    Chemins.warn(f"Condition violated by node {v}.", "cyan")
                    return False

        for k in self.list.keys():
            if list(self.list[k].keys())[0] != k:
                Chemins.warn(f"C3.b: Each employee should visits their home.")
                Chemins.warn(f"Condition violated by employee {k}.", "cyan")
                return False

        # C4, okay thank to the format 

        # C5 + C6.b
        for k in self.list.keys():
            delta1 = 0
            delta2 = 0
            for v in self.list[k].keys(): 
                if v == k :
                    continue
                self.list[k][v][0] += delta1
                self.list[k][v][1] -= delta2

                opening, closing = Node.list[v].opening_time, Node.list[v].closing_time
                m = self.list[k][v][1] - opening
                if m<0 :
                    Chemins.warn(f"C5: A task should be worked on between its opening time and closing time")
                    Chemins.warn(f"Condition violated by task {v}", "cyan")
                    return False
                if  opening > self.list[k][v][0] :
                    delta1 += opening - self.list[k][v][0]
                    self.list[k][v][0] = opening
                
                m = closing - (self.list[k][v][0] + Node.list[v].duration)
                if m<0 :
                    Chemins.warn(f"C5: A task should be worked on between its opening time and closing time")
                    Chemins.warn(f"Condition violated by task {v}", "cyan")
                    return False
                if  closing < self.list[k][v][1] + Node.list[v].duration:
                    delta2 += (self.list[k][v][1] + Node.list[v].duration) - closing
                    self.list[k][v][1] = closing - Node.list[v].duration

                if v in unavails :
                    self.list[k][v][0] = opening
                    self.list[k][v][1] = opening

        # C6.a + 
        for k in self.list.keys():
            if len(list(self.list[k].keys())) == 1: continue
            v = list(self.list[k].keys())[1]
            if v not in unavails :
                if self.list[k][v][0] < Employee.list[k].start_time :
                    Chemins.warn(f"C6.a: Employees' start times should be respected")
                    Chemins.warn(f"Condition violated by employee {k}", "cyan")
                    return False

        # C7
        # TODO: verify C7

        # C8
        # TODO: verify C8

        # C9
        for k in self.list.keys():
            if len(list(self.list[k].keys())) == 1 : continue
            v = list(self.list[k].keys())[-1]
            if self.list[k][v][1] + Node.list[v].duration + (Node.distance[v, k] / Employee.speed) > Employee.list[k].end_time :
                Chemins.warn("C9: An employee should have enough time to go back home.")
                Chemins.warn(f"Condition violated by employee {k} after task {v}", "cyan")
                return False

        # C10
        for k in self.list.keys() :
            Lunch_Break = False
            for i in range(len(self.list[k].keys())):
                v = list(self.list[k].keys())[i]
                t1,t2 = self.list[k][v]
                if t1<= 13*60 and t2>=13*60 and t2-t1>=60 :
                    if t1<=12*60:
                        t_end_lunch = 13*60
                    else :
                        t_end_lunch = t1+60
                    def rec(i,t):
                        if i==len(self.list[k].keys()):
                            return True
                        else :
                            v1 = list(self.list[k].keys())[i-1]
                            v2 = list(self.list[k].keys())[i]
                            t1,t2 = self.list[k][v2]
                            if v1 == k:
                                d = (Node.distance[v1,v2]/Employee.speed)
                            else :
                                d = Node.list[v1].duration + (Node.distance[v1,v2]/Employee.speed)
                            T = t + d
                            if T>t2:
                                return False
                            if T<=t1 :
                                return True
                            else :
                                return rec(i+1,T)
                    Lunch_Break = rec(i+1,t_end_lunch)
                    if Lunch_Break : break
            
            if v>=len(self.list.keys()) and t1 + Node.list[v].duration <= 13*60 and Employee.list[k].end_time:
                if Employee.list[k].end_time >= max(t1 + Node.list[v].duration,12*60) + 60 +Node.distance[v,k]/Employee.speed :
                    break
                    
            if not Lunch_Break :
                Chemins.warn("C10: Each employee must have the time to lunch")
                Chemins.warn(f"Condition violated by employee {k}", "cyan")
                return False

        # C12
        for k in self.list.keys():
            for i in range(len(self.list[k].keys())-1):
                v1 = list(self.list[k].keys())[i]
                v2 = list(self.list[k].keys())[i+1]
                if i == 0 and self.list[k][v1][0] + (Node.distance[v1, v2] / Employee.speed) > self.list[k][v2][1] :
                    Chemins.warn("C12: The traveling time between two nodes should be sufficient.")
                    Chemins.warn(f"Condition violated between node {v1} and {v2}", "cyan")
                    return False

        # C13
        for k in self.list.keys():
            for v in self.list[k].keys():
                if v in tasks and Employee.list[k].level < Node.list[v].level :
                    Chemins.warn("C13: An employee can only do tasks that they are able to do.")
                    Chemins.warn(f"Condition violated by employee {k} at node {v}", "cyan")
                    return False

        # Every condition is verified
        return True
    





# Demo
On peut instancier une nouvelle instance de Solution en appelant simplement le constructeur Solution().
L’instance s’adapte alors automatiquement aux données présentes dans les classes Employee et Node

La méthode validate permet de déterminer si notre solution vérifie les contraintes. Lorsque au moins une contrainte n’est pas vérifiée. Dans ce cas-là, un message d’erreur correspondant à la première violation de contrainte s’affiche.

In [7]:
#solution_instance = Solution()
#solution_instance.validate()

Pour modifier une solution, on modifie directement les tableaux numpy stockés en attribut des instances.

In [8]:
#solution_instance.x[0, 0] = 1
#solution_instance.validate()

Pour copier une instance, on utilise la méthode copy(), qui fait un deep-copy de l’instance.

In [9]:
#solution_instance_copy = solution_instance.copy()

Lorsque l’on modifie une copie, la solution originale n’est pas modifiée.

In [10]:
#solution_instance_copy.x[0, 0] = 2
#solution_instance.x[0, 0] = 0
#solution_instance.x[1, 0] = solution_instance.x[0, 2] = 1

In [11]:
#solution_instance.validate()

In [12]:
#solution_instance_copy.validate()

L'affichage des contraintes non vérifiées sont pour le debug, et on peut ne pas les afficher lorsque l’on exécute nos algorithmes méta-heuristiques.

In [13]:
#Solution.set_warning(False) # this tells the solution class to not print warning messages
#solution_instance.validate()

In [14]:
#Solution.set_warning(True) # modify setting to print warning messages again
#solution_instance.validate()

In [35]:
# Tabu Search

# Définition des objets

def evaluate(routes):
    total_task_duration = 0
    total_distance = 0
    for route in routes :
        for i in range(1,len(route)) :
            if route[i] not in unavails :
                total_task_duration += Node.list[route[i]].duration
            total_distance += Node.distance[route[i-1],route[i]]
        total_distance += Node.distance[route[-1],route[0]]
    return (total_task_duration, total_distance)

def compare(a,b):
    if a[0] < b[0] :
        return True
    elif a[0] == b[0] and a[1] > b[1] :
        return True
    return False

def simple_sol(route):
    sol =[]
    for i in range(len(Employee.list)):
        sol.append([i])
    for v in unavails :
        employee = Node.list[v].employee
        employee_idx = Employee.index_of(employee)
        sol[employee_idx].append(v)
    sol[route[0]] = route
    return sol

def add_time(routes):
    routes_timed = {}
    for k in range(len(routes)):
        routes_timed[k] = {}
        v0 = routes[k][0]
        routes_timed[k][v0] = Employee.list[k].start_time
        for i in range(1,len(routes[k])):
            v1 = routes[k][i-1]
            v2 = routes[k][i]
            if i == 1 :
                d = ceil(Node.distance[v1,v2]/Employee.speed)
            else :
                if v1<len(Employee.list):
                    return "Not Possible"
                d = Node.list[v1].duration + ceil(Node.distance[v1,v2]/Employee.speed)
            routes_timed[k][v2] = routes_timed[k][v1] + d
        
        v = routes[k][-1]
        if v<len(routes):
            margin = Employee.list[k].end_time - (routes_timed[k][v] + ceil(Node.distance[v,v0]/Employee.speed))
        else :
            margin = Employee.list[k].end_time - (routes_timed[k][v] + Node.list[v].duration + ceil(Node.distance[v,v0]/Employee.speed))
        if margin <0 :
            return "Not Possible"
        else :
            for i in range(len(routes[k])):
                v = routes[k][i]
                routes_timed[k][v] = [routes_timed[k][v],routes_timed[k][v]+margin]
    return routes_timed


# Définition de la list Tabou

def update_tabu(Tabu_list, opp, iter, tabu_step):
    Tabu_list[iter] = opp
    if iter >= tabu_step :
        del Tabu_list[iter - tabu_step]

# Définition du voisinage

def Crossing(route1, route2, max_len_cross):
    Neighbors = []
    n1 = len(route1)
    n2 = len(route2)
    for l1 in range(max_len_cross+1):
        for l2 in range(max_len_cross+1): 
            if l1 == 0 and l2 ==0:
                continue
            for i in range(1,n1-l1):
                for j in range(1,n2-l2):
                    new_route1 = route1[:i]+route2[j:j+l2]+route1[i+l1:]
                    new_route2 = route2[:j]+route1[i:i+l1]+route2[j+l2:]
                    Neighbors.append((new_route1,new_route2))
    return Neighbors

def Crossing_alone(route):
    Neighbors = []
    permutations = list(itertools.permutations(route[1:]))
    for p in permutations :
        Neighbors.append([route[0]]+[x for x in p])
    return Neighbors

def Adding_Node(route,unvisited_nodes):
    Neighbors = []
    for node in unvisited_nodes :
        for i in range(len(route)):
                Neighbors.append((route[:i+1]+[node]+route[i+1:],node))
    return Neighbors

def Deleting_Node(route):
    Neighbors = []
    for i in range(1,len(route)):
        if route[i] not in unavails:
            Neighbors.append((route[:i]+route[i+1:], route[i]))
    return Neighbors

def local_neighbor_switching(route):
    candidate_list = Crossing_alone(route)
    local_obj_values = (0,0)
    for candidate in candidate_list:
        if Chemins(add_time(simple_sol(candidate))).validate() :
            obj_values = evaluate([candidate])
            if compare(local_obj_values, obj_values) :
                local_sol = candidate
                local_obj_values = obj_values

    return local_sol

def local_neighbor_adding(route,unvisited_nodes):
    candidate_list = Adding_Node(route,unvisited_nodes)
    local_obj_values = (0,0)
    add_node = -1
    for candidate, node in candidate_list:
        if Chemins(add_time(simple_sol(candidate))).validate() :
            obj_values = evaluate([candidate])
            if compare(local_obj_values, obj_values) :
                local_sol = candidate
                local_obj_values = obj_values
                add_node = node

    if add_node == -1:
        return route

    del unvisited_nodes[add_node]

    return local_sol

def create_neighborhood(routes, unvisited_nodes, status, max_len_cross, Tabu_list):
    Neighborhood = []
    T = len(routes)

    if status == "adding":
        new_route = []
        for i in range(T):
            Neighbor = local_neighbor_adding(routes[i],unvisited_nodes)
            new_route.append(Neighbor)
        Neighborhood = [new_route]
        
    elif status == "switching":
        new_route = []
        for i in range(T):
            Neighbor = local_neighbor_switching(routes[i],unvisited_nodes)
            new_route.append(Neighbor)
        Neighborhood = [new_route]

    elif status == "deleting":
        local_obj_values = (0,0)
        del_node = -1
        for i in range(T):
            Neighbors = Deleting_Node(routes[i])
            for candidate, node in Neighbors:
                new_routes = routes[:i]+[candidate]+routes[i+1:]
                obj_values = evaluate([candidate])
                if compare(local_obj_values, obj_values) :
                    local_sol = new_routes
                    local_obj_values = obj_values
                    del_node = node
        
        if del_node == -1:
            Neighborhood = [routes]
        else :
            unvisited_nodes[del_node] = None
            Neighborhood = [local_sol]
        
    else :
        for i in range(T):
            for j in range(i+1,T):
                Neighbors = Crossing(routes[i], routes[j], max_len_cross)
                for (route1,route2) in Neighbors:
                    new_routes = routes[:i]+[route1]+routes[i+1:j]+[route2]+routes[j+1:]
                    if Chemins(add_time(new_routes)).validate() and new_routes not in Tabu_list:
                        Neighborhood.append(new_routes)
    return Neighborhood

# Initialisation / 1ère solution
def sol_glouton():
    sol =[]
    for i in range(len(Employee.list)):
        sol.append([i])
    for v in unavails :
        employee = Node.list[v].employee
        employee_idx = Employee.index_of(employee)
        sol[employee_idx].append(v)
    return sol

def search_unvisited_nodes(sol):
    unvisited_nodes = {}
    for i in range(len(Node.list)):
        unvisited_nodes[i] = None
    for route in sol:
        for node in route :
            del unvisited_nodes[node]
    return unvisited_nodes

# Itérations

def tabu_search(max_len_cross,tabu_step,max_it,block_max):
    sol = sol_glouton()
    sol_list = [sol]
    best_obj_values = evaluate(sol)
    Tabu_list = {0 : sol}
    block_count = 0
    unvisited_nodes = search_unvisited_nodes(sol)
    climbing = True
    for it in range(1,max_it):
        block_count += 1
        if block_count >= block_max//2 :
            climbing = True

        if block_count >= block_max :
            neighborhood = create_neighborhood(sol_list[-1], unvisited_nodes, "deleting", max_len_cross, Tabu_list)
            block_count = 0
        elif climbing :
            neighborhood = create_neighborhood(sol_list[-1], unvisited_nodes, "adding", max_len_cross,Tabu_list)
            if neighborhood[0] == [sol_list[-1]] :
                climbing = False
                neighborhood = create_neighborhood(sol_list[-1], unvisited_nodes, "switching", max_len_cross, Tabu_list)
                neighborhood = neighborhood + create_neighborhood(sol_list[-1], unvisited_nodes, " ", max_len_cross, Tabu_list)
        else :
            neighborhood = create_neighborhood(sol_list[-1], unvisited_nodes, "switching", max_len_cross, Tabu_list)
            neighborhood = neighborhood + create_neighborhood(sol_list[-1], unvisited_nodes, " ", max_len_cross, Tabu_list)

        local_obj_values = (0,0)
        for candidate in neighborhood :
            obj_values = evaluate(candidate)
            if compare(local_obj_values, obj_values) :
                local_sol = candidate
                local_obj_values = obj_values
        if local_obj_values == (0,0):
            return "No avaliable Neighbor"

        update_tabu(Tabu_list, local_sol, it, tabu_step)
        sol_list.append(local_sol)

        if compare(best_obj_values, local_obj_values):
            sol = local_sol
            best_obj_values = local_obj_values
            block_count = 0

        print(it, local_obj_values)
    return sol, best_obj_values


In [16]:
Chemins.set_warning(False)
#create_neighborhood([[0,10,7,4,6,12],[1,5,8,11,9]], [3],10)

print(sol_glouton())

Chemins(add_time(sol_glouton())).validate()

[[0, 118], [1], [2], [3, 119], [4], [5], [6], [7], [8, 120], [9]]


True

In [37]:
#Bordeaux V2 : 1,100,200,20
tabu_search(3,20,50,2)

1 (990, 1579322.6561589)
2 (1465, 2125622.931290979)
3 (1845, 2352608.181074199)
4 (2135, 2392136.326361952)
5 (2255, 2434614.68784876)
6 (2335, 2448297.6978071593)
7 (2375, 2451528.346879376)
8 (2375, 2451528.346879376)
9 (2335, 2448297.6978071593)
10 (2375, 2451528.346879376)
11 (2335, 2448297.6978071593)
12 (2375, 2451528.346879376)
13 (2335, 2448297.6978071593)
14 (2375, 2451528.346879376)
15 (2335, 2448297.6978071593)
16 (2375, 2451528.346879376)
17 (2335, 2448297.6978071593)
18 (2375, 2451528.346879376)
19 (2335, 2448297.6978071593)
20 (2375, 2451528.346879376)
21 (2335, 2448297.6978071593)
22 (2375, 2451528.346879376)
23 (2335, 2448297.6978071593)
24 (2375, 2451528.346879376)
25 (2335, 2448297.6978071593)
26 (2375, 2451528.346879376)
27 (2335, 2448297.6978071593)
28 (2375, 2451528.346879376)
29 (2335, 2448297.6978071593)
30 (2375, 2451528.346879376)
31 (2335, 2448297.6978071593)
32 (2375, 2451528.346879376)
33 (2335, 2448297.6978071593)
34 (2375, 2451528.346879376)
35 (2335, 244

([[0, 118, 64, 60, 63, 82],
  [1, 24, 27, 117, 101, 114, 111],
  [2, 10, 102, 110],
  [3, 97, 52, 113, 119],
  [4, 28, 11],
  [5, 32, 30, 35, 42],
  [6, 18, 37, 106, 25, 26],
  [7, 89, 92, 88, 16],
  [8, 33, 12, 120, 46, 38, 17, 100, 36],
  [9, 19, 112, 39, 29]],
 (2375, 2451528.346879376))

In [31]:
#evaluate([[0, 10, 7, 4, 6, 12], [1, 3, 5, 8, 11, 9]])